In [26]:
import scipy.io
import mat73
import pandas as pd
import numpy as np
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings

# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)

import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()):
    if "anuja" in os.environ.get('USER'):
        DATA_DIR = 'data/'
    #elif "martyna" in os.environv.get('USER'):
    #    DATA_DIR = '/home/ubuntu/Martyna/repo/AI4Health/'

In [27]:
#Martyna  
        DATA_DIR = '/home/ubuntu/Martyna/repo/AI4Health/DATAfoof/'

## EEG data and foof:

In [28]:
## data loading
data = scipy.io.loadmat(DATA_DIR+'x.mat')  
foof = pd.read_csv(DATA_DIR+"sensor_intercept_slope.csv")
foof

,0,1,2,3,4,5,6,7,8,9,...,201,202,203,204,205,206,207,208,209,IDs
0,1.610769,1.793923,1.639958,1.803579,1.622763,1.867163,1.596286,1.868653,1.459489,1.880624,...,1.637747,1.472414,1.775967,1.534612,1.803604,1.575800,1.869977,0.986272,1.825774,NDARAA075AMK
1,1.572269,1.762184,1.615888,1.938907,1.515331,1.878222,1.577389,1.888830,1.645799,1.892993,...,1.698141,1.698537,1.854421,1.486611,1.755794,1.517099,1.845983,1.486650,1.888544,NDARAA112DMH
2,1.414507,1.818785,1.504383,1.902537,1.563395,1.936381,1.554663,2.085351,1.566065,2.106747,...,1.847331,1.417519,1.869143,1.323095,1.838333,1.496008,1.895424,1.593155,2.095749,NDARAA117NEJ
3,0.561062,1.068994,0.333554,0.991180,0.555779,1.304169,0.665864,1.491464,0.524828,1.573271,...,1.464311,0.576556,1.265274,0.199181,0.863242,0.246682,1.010393,0.703331,1.724831,NDARAA947ZG5
4,1.262007,1.901401,1.305927,1.924721,1.293914,1.924840,1.226456,1.947274,0.818595,1.905113,...,1.859064,1.158560,1.863190,1.173287,1.773532,1.252168,1.878925,0.918020,1.749441,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,0.648706,1.508826,0.591809,1.640814,0.640782,1.665891,0.455759,1.679019,0.866016,1.786340,...,1.536850,0.528776,1.478037,0.466827,1.767683,0.570274,1.664126,1.351549,1.996940,NDARZN277NR6
2038,1.332278,1.729382,1.387863,1.788208,1.452167,1.841329,1.524229,1.976239,1.480653,2.022584,...,1.876485,1.214977,1.795080,1.456392,1.835807,1.399127,1.795906,1.380795,2.036327,NDARZN578YDP
2039,0.746123,1.324138,0.653335,1.285734,0.564490,1.271931,0.375903,1.245331,0.258508,1.206112,...,1.307891,0.691917,1.310004,0.604437,1.283587,0.564744,1.273098,0.339229,1.050644,NDARZN610GTY
2040,0.976055,1.441830,0.823197,1.407064,0.879368,1.464799,0.902699,1.512205,1.012450,1.505105,...,1.341166,0.598687,1.282644,0.697733,1.421177,0.753861,1.457204,0.781225,1.470061,NDARZN677EYE


In [5]:
# flattening
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df = np.array(df)
df = df.reshape(data['x'].shape)
df.shape

(2042, 105, 79)

In [6]:
df = pd.DataFrame(df.reshape((df.shape[0], -1)))
df['IDs'] = foof['IDs']
df

,0,1,2,3,4,5,6,7,8,9,...,8286,8287,8288,8289,8290,8291,8292,8293,8294,IDs
0,7.707834e-12,1.512286e-10,2.475735e-09,3.453763e-08,4.554996e-07,0.000007,0.000109,0.001314,0.009992,0.045200,...,2.990881e-09,1.223086e-09,4.893694e-10,1.915752e-10,7.337775e-11,2.749867e-11,1.008282e-11,3.617107e-12,1.269651e-12,NDARAA075AMK
1,3.798713e-02,1.639433e-01,2.897591e-01,2.097609e-01,6.347556e-02,0.027758,0.105298,0.184513,0.130638,0.163258,...,5.835291e-06,1.856717e-06,5.922864e-07,1.928109e-07,6.449918e-08,2.204650e-08,7.598678e-09,2.604260e-09,8.780452e-10,NDARAA112DMH
2,2.104310e-06,4.299647e-05,5.644195e-04,4.760137e-03,2.579253e-02,0.089799,0.201005,0.290260,0.276177,0.196151,...,1.332064e-07,5.270957e-08,2.026971e-08,7.575282e-09,2.751336e-09,9.711398e-10,3.331297e-10,1.110552e-10,3.597966e-11,NDARAA117NEJ
3,1.692639e-01,2.021357e-01,2.273063e-01,2.406964e-01,2.400039e-01,0.225430,0.202278,0.198154,0.228356,0.181836,...,8.323825e-02,7.759521e-02,7.183415e-02,6.604061e-02,6.029416e-02,5.466678e-02,4.922161e-02,4.401211e-02,3.908162e-02,NDARAA947ZG5
4,8.332328e-06,1.552379e-04,1.804338e-03,1.287033e-02,5.615280e-02,0.149809,0.244887,0.247738,0.163687,0.094061,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,8.241315e-06,2.938160e-04,4.641777e-03,3.249546e-02,1.008094e-01,0.138615,0.084707,0.024339,0.009157,0.021386,...,5.852474e-03,4.262113e-03,3.068545e-03,2.184048e-03,1.536788e-03,1.069025e-03,7.351630e-04,4.998062e-04,3.359246e-04,NDARZN277NR6
2038,8.113400e-03,1.005675e-02,1.237930e-02,1.513279e-02,1.837070e-02,0.022147,0.026515,0.031525,0.037230,0.043721,...,5.855173e-04,3.933072e-04,2.601868e-04,1.695084e-04,1.087534e-04,6.871244e-05,4.275280e-05,2.619553e-05,1.580597e-05,NDARZN578YDP
2039,5.472767e-03,8.816325e-03,1.629356e-02,3.226822e-02,6.212373e-02,0.108594,0.166461,0.220594,0.252412,0.252807,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARZN610GTY
2040,2.658455e-03,5.749806e-03,1.180538e-02,2.300961e-02,4.257369e-02,0.074778,0.124685,0.197358,0.296551,0.423009,...,3.676327e-04,1.925477e-04,9.805386e-05,4.855040e-05,2.337344e-05,1.094093e-05,4.979521e-06,2.203549e-06,9.481114e-07,NDARZN677EYE


## Behavioral Data:


In [7]:
behaviour_data = pd.read_csv(DATA_DIR+'AllData.csv')
behaviour_data = behaviour_data.rename(columns = {'EID': 'IDs'}, inplace = False)
behaviour_data.columns

Index(['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_SR_GD', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum'],
      dtype='object')

In [8]:
behaviour_data = behaviour_data[['IDs', 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
       'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
       'WISC_VCI_Sum']]
behaviour_data.shape

(2579, 9)

In [9]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns (with column 'IDs').\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(behaviour_data)

Your selected dataframe has 9 columns (with column 'IDs').
There are 8 columns that have missing values.


,Missing Values,% of Total Values
WISC_WMI_Sum,712,27.6
WISC_VCI_Sum,702,27.2
SCARED_P_GD,576,22.3
SRS_SCI_T,448,17.4
SRS_RRB_T,447,17.3
SWAN_IN_Avg,385,14.9
SWAN_HY_Avg,385,14.9
SWAN_Avg,385,14.9


## Merge EEG and foof data together:

In [10]:
data = pd.merge(df, foof, on='IDs', how='inner')
data.shape

(2042, 8506)

## and then with the behaviorals

In [11]:
df = pd.merge(data, behaviour_data, on='IDs', how='inner')
df.shape

(1465, 8514)

## features and labels preparation:

In [12]:
#removing NaNs
df = df.dropna()
df.shape

(1075, 8514)

In [13]:
labels_list = [ 'SRS_SCI_T', 'SRS_RRB_T', 'SWAN_IN_Avg', 'SWAN_HY_Avg',
   'SWAN_Avg', 'SCARED_P_GD', 'WISC_WMI_Sum',
   'WISC_VCI_Sum']

X = df[df.columns.difference(['IDs']+labels_list)]
print(X.shape)

(1075, 8505)


In [14]:
Y = df[labels_list]
print(Y.shape)

(1075, 8)


## Split and PCA



In [15]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.25, shuffle=True)

In [16]:
# scaling x
norm = preprocessing.MinMaxScaler().fit(xtrain)

# transform training data
xtrain = norm.transform(xtrain)
xtest = norm.transform(xtest)
print(xtrain.shape, xtest.shape)

print('Applying PCA...')
pca = PCA(.90) # 95% variance retained
pca.fit(xtrain)

# transform data
xtrain = pca.transform(xtrain)
xtest = pca.transform(xtest)
print(xtrain.shape, xtest.shape)

(806, 8505) (269, 8505)
Applying PCA...
(806, 96) (269, 96)


In [17]:
# scaling y
min_max_scaler = preprocessing.MinMaxScaler().fit(ytrain)
ytrain = min_max_scaler.transform(ytrain)
ytest = min_max_scaler.transform(ytest)

print(ytrain.shape, ytest.shape)

(806, 8) (269, 8)


In [18]:
ytrain.mean(axis=0)

array([0.41193365, 0.33258216, 0.61844036, 0.53768034, 0.58896723,
       0.26550868, 0.45758375, 0.4943074 ])

## MODELLING PART

### dummy regressors (to obtain the random baseline):

In [19]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.dummy import DummyRegressor


lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(xtrain, ytrain)
lm_dummy_median = DummyRegressor(strategy = 'median').fit(xtrain, ytrain)
ypred_dummy_mean = lm_dummy_mean.predict(xtest)
ypred_dummy_median = lm_dummy_median.predict(xtest)


print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(ytest,
																	ypred_dummy_mean)))

print("Median absolute error (dummy): {:.2f}".format(median_absolute_error(ytest,
																	ypred_dummy_median)))

print("r2_score (dummy mean): {:.2f}".format(r2_score(ytest, ypred_dummy_mean)))
print("r2_score (dummy median): {:.2f}".format(r2_score(ytest, ypred_dummy_median)))

print(ytest[:10])
print(Y.mean())

Mean squared error (dummy): 0.04
Median absolute error (dummy): 0.13
r2_score (dummy mean): -0.00
r2_score (dummy median): -0.02
[[0.51851852 0.40816327 0.6296295  0.66666667 0.66037731 0.33333333
  0.28125    0.38235294]
 [0.7037037  0.55102041 0.77777767 0.72222217 0.76415106 0.16666667
  0.40625    0.47058824]
 [0.74074074 0.71428571 0.425926   0.6296295  0.53773582 0.22222222
  0.78125    0.55882353]
 [0.72222222 0.69387755 0.72222217 0.4444445  0.59433971 0.55555556
  0.125      0.38235294]
 [0.62962963 0.51020408 0.85185183 0.85185183 0.86792464 0.44444444
  0.09375    0.17647059]
 [0.37037037 0.18367347 0.5555555  0.611111   0.59433971 0.38888889
  0.25       0.38235294]
 [0.12962963 0.08163265 0.5        0.5        0.50943404 0.05555556
  0.65625    0.58823529]
 [0.31481481 0.10204082 0.648148   0.79629617 0.73584911 0.
  0.5        0.58823529]
 [0.48148148 0.18367347 0.85185183 0.75925917 0.82075478 0.05555556
  0.46875    0.14705882]
 [0.61111111 0.40816327 0.5185185  0.59259

## SVR regressor

In [20]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Create the SVR regressor
svr = SVR(epsilon=0.2)
#Create the Multioutput Regressor
mor = MultiOutputRegressor(svr)
# Train the regressor
mor = mor.fit(xtrain, ytrain)
# Generate predictions for testing data
ypred = mor.predict(xtest)


print("Mean squared error (SVC): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (SVC): {:.2f}".format(mor.score(xtest, ytest)))


Mean squared error (SVC): 0.04
R2 score (SVC): -0.07


## Linear Regression

In [21]:
# define model
model = LinearRegression()
# fit model
model.fit(xtrain, ytrain)

# Generate predictions for testing data
ypred = model.predict(xtest)

print("Mean squared error (LinReg): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (LinReg): {:.2f}".format(model.score(xtest, ytest)))												



Mean squared error (LinReg): 0.04
R2 score (LinReg): -0.06


## Random forest regressor

In [22]:
max_depth = 30
regr_rf = RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
regr_rf.fit(xtrain, ytrain)
# Predict on new data
y_rf = regr_rf.predict(xtest)
regr_rf.score(xtest, ytest)



print("Mean squared error (RandomForest): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (RandomForest): {:.2f}".format(regr_rf.score(xtest, ytest)))


Mean squared error (RandomForest): 0.04
R2 score (RandomForest): -0.00


 ## multi-output meta estimator

In [23]:
max_depth = 30
regr_multirf = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=100, max_depth=max_depth, random_state=7)
)
regr_multirf.fit(xtrain, ytrain)
y_multirf = regr_multirf.predict(xtest)


print("Mean squared error (regr_multirf): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))
print("R2 score (regr_multirf): {:.2f}".format(regr_multirf.score(xtest, ytest)))

Mean squared error (regr_multirf): 0.04
R2 score (regr_multirf): -0.01


## regressor chains

In [24]:
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import RegressorChain
from sklearn.svm import LinearSVR
# define dataset
# define base model
model = LinearSVR()
# define the chained multioutput wrapper model
wrapper = RegressorChain(model)
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(wrapper, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance

print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int

MAE: 5.547 (0.298)


/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


## mlp with keras

In [25]:
in_dim = xtrain.shape[1]
out_dim = ytrain.shape[1]


model = Sequential()
model.add(Dense(64, input_dim=in_dim, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(64, input_dim=in_dim, activation="relu"))

model.add(Dense(out_dim))
model.compile(loss="mse", metrics = ['mse'], optimizer="adam")


model.fit(xtrain, ytrain, epochs=100, batch_size=12, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)

score = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)
score= np.array(score)
score.mean()

2021-12-06 16:01:35.009555: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-12-06 16:01:35.032840: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 16:01:35.033530: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:06.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-12-06 16:01:35.033745: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-12-06 16:01:35.035122: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-12-06 16:01:35.036584: I tensorflow/stream_executor/platform/default/d

Epoch 1/100
 1/54 [..............................] - ETA: 0s - loss: 0.4397 - mse: 0.4397

2021-12-06 16:01:35.991329: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


54/54 [==============================] - 0s 6ms/step - loss: 0.1516 - mse: 0.1516 - val_loss: 0.0869 - val_mse: 0.0869
Epoch 2/100
54/54 [==============================] - 0s 2ms/step - loss: 0.0660 - mse: 0.0660 - val_loss: 0.0697 - val_mse: 0.0697
Epoch 3/100
54/54 [==============================] - 0s 2ms/step - loss: 0.0526 - mse: 0.0526 - val_loss: 0.0637 - val_mse: 0.0637
Epoch 4/100
54/54 [==============================] - 0s 2ms/step - loss: 0.0460 - mse: 0.0460 - val_loss: 0.0599 - val_mse: 0.0599
Epoch 5/100
54/54 [==============================] - 0s 2ms/step - loss: 0.0414 - mse: 0.0414 - val_loss: 0.0582 - val_mse: 0.0582
Epoch 6/100
54/54 [==============================] - 0s 2ms/step - loss: 0.0379 - mse: 0.0379 - val_loss: 0.0584 - val_mse: 0.0584
Epoch 7/100
54/54 [==============================] - 0s 2ms/step - loss: 0.0352 - mse: 0.0352 - val_loss: 0.0583 - val_mse: 0.0583
Epoch 8/100
54/54 [==============================] - 0s 2ms/step - loss: 0.0328 - mse: 0.0328 -

0.08003305